# Getting Started with Spark & Iceberg

**Important:** Spark session is **automatically created** when this notebook opens.

The `spark` variable is ready to use immediately — no setup needed!

## Part 1: Spark Session (Already Running!)

Your Spark session is created automatically by the PySpark Jupyter container.

Just run the cell below to verify it:

In [ ]:
# The spark session is automatically created in PySpark Jupyter
# Let's verify it's working

print("✓ Spark Session Active")
print(f"Spark Version: {spark.version}")
print(f"App Name: {spark.sparkContext.appName}")
print(f"Master: {spark.sparkContext.master}")

## Part 2: Verify Iceberg & Polaris Configuration

Your Spark is configured to use Polaris as the catalog. Let's check:

In [ ]:
# Check Spark configuration

print("=== Catalog Configuration ===")
print(f"Default Catalog: {spark.conf.get('spark.sql.defaultCatalog')}")
print(f"Polaris URI: {spark.conf.get('spark.sql.catalog.polaris.uri')}")
print(f"S3 Endpoint: {spark.conf.get('spark.hadoop.fs.s3a.endpoint')}")
print()
print("✓ Everything configured correctly!")

## Part 3: Create Your First Namespace (Schema)

A namespace is like a database or schema. Let's create one:

In [ ]:
# Create a namespace (schema) for your tables
# This is just like CREATE DATABASE in traditional SQL

spark.sql("CREATE NAMESPACE IF NOT EXISTS my_warehouse")

print("✓ Namespace created: my_warehouse")

# Verify it exists
spark.sql("SHOW NAMESPACES IN polaris").show()

## Part 4: Create Your First Iceberg Table

Now create a simple table to store customer data:

In [ ]:
# Create an Iceberg table
# USING ICEBERG makes it an Iceberg table (not Delta)
# PARTITIONED BY helps with query performance

spark.sql("""
    CREATE TABLE IF NOT EXISTS my_warehouse.customers (
        customer_id INT,
        name STRING,
        email STRING,
        country STRING,
        created_at TIMESTAMP
    )
    USING ICEBERG
    PARTITIONED BY (CAST(created_at AS DATE))
""")

print("✓ Table created: my_warehouse.customers")

# Show table schema
spark.sql("DESCRIBE TABLE my_warehouse.customers").show()

## Part 5: Insert Sample Data

Let's add some rows to the table:

In [ ]:
# Insert sample data

spark.sql("""
    INSERT INTO my_warehouse.customers VALUES
    (1, 'Alice Johnson', 'alice@example.com', 'USA', '2025-01-15 10:30:00'),
    (2, 'Bob Smith', 'bob@example.com', 'Canada', '2025-01-16 14:20:00'),
    (3, 'Charlie Brown', 'charlie@example.com', 'USA', '2025-01-17 09:15:00'),
    (4, 'Diana Prince', 'diana@example.com', 'UK', '2025-01-18 11:45:00'),
    (5, 'Eve Wilson', 'eve@example.com', 'USA', '2025-01-19 16:30:00')
""")

print("✓ Inserted 5 rows")

## Part 6: Query Your Data

The query pattern is simple: `schema.tablename`

In [ ]:
# Simple query - just use schema.tablename
# No catalog prefix needed because polaris is the default catalog

spark.sql("SELECT * FROM my_warehouse.customers").show()

## Part 7: Filter Your Data

In [ ]:
# Filter data with WHERE clause

spark.sql("""
    SELECT name, email, country 
    FROM my_warehouse.customers 
    WHERE country = 'USA'
""").show()

## Part 8: Use DataFrame API (Alternative)

You can also use the DataFrame API instead of SQL:

In [ ]:
# Load table into a DataFrame
df = spark.table("my_warehouse.customers")

# Filter and select columns
df.filter(df.country == "USA").select("name", "email").show()

## Part 9: Verify Data Persists

Your data is stored in MinIO and will persist even if you stop containers.
Let's check the table metadata:

In [ ]:
# View table metadata and history

print("=== Table Info ===")
spark.sql("SHOW TABLES IN my_warehouse").show()

print("\n=== Row Count ===")
spark.sql("SELECT COUNT(*) as total_rows FROM my_warehouse.customers").show()

print("\n✓ Data is persisted in MinIO and PostgreSQL")
print("  - Even if you stop/restart containers, this data survives!")

## Part 10: View Data in MinIO Console

Your actual data files are stored in MinIO. To see them:

1. Go to http://localhost:9001
2. Login: `minioadmin` / `minioadmin`
3. Navigate: `warehouse` → `polaris` → `my_warehouse` → `customers` → `data`
4. You'll see `.parquet` files with your data!

The metadata about these files is stored in PostgreSQL (via Polaris catalog).

## Summary

✅ **Spark Session**: Created automatically (already done!)

✅ **Iceberg Table**: Created with `CREATE TABLE ... USING ICEBERG`

✅ **Data Inserted**: Used SQL INSERT statement

✅ **Data Queried**: Simple `schema.tablename` pattern

✅ **Data Persists**: Stored in MinIO + PostgreSQL

### Next Steps:

1. **Create more tables** for your data
2. **Join tables** together
3. **Explore advanced features** like time travel in the example notebook
4. **Add your own data** by creating CSV/Parquet files in `workspace/data/`

### Key Takeaway:

**You DON'T need to create a Spark session manually** — it's already created for you in PySpark Jupyter! Just use the `spark` variable directly.